# This is the current leading entry in the leaderboard of the image-woof challenge. 

## How to make this work
  - remove "dw=self.dw, se=self.se" in model_constructor/net.py:118
  - save those files
  - restart and run all
  - _To make this easier, we could actually copy the module to our repo and modify it there_

# ResNet with a Twist, with depthwise separable convolution

> depthwise (x4) + Ranger + Mish + SA + MaxBlurPool + ResTrick

See summary at https://forums.fast.ai/t/imagenette-imagewoof-leaderboards/45822/47?u=liuyao 

## Imagewoof Leaderboard

Imagewoof2, with a 70/30 train/test ratio.

| Size (px) | Epochs | x2 | x4 | x6 | x4 double | runs |
|--|--|--| --|--| --|--|
|128|5|75.19|76.27|**76.61**| **82.12**|5, mean
|128|20|85.18|86.22|**86.27**| **88.93**|5, mean
|128|80|87.70|**87.83**|87.65| **90.15**|1
|128|200|
|192|5|79.86|**81.15**|| **82.69**|5, mean
|192|20|88.12|**88.37**|
|192|80|**90.30**|89.89|| **92.08** |
|192|200|
|256|5|
|256|20|
|256|80|
|256|200|


# setup and imports

In [ ]:
!pip install git+https://github.com/ayasyrev/model_constructor
!pip install fastai==1.0.61
# !pip install git+https://github.com/fastai/fastai.git

  Cloning https://github.com/ayasyrev/model_constructor to /tmp/pip-req-build-d9c7zi3u
  Running command git clone -q https://github.com/ayasyrev/model_constructor /tmp/pip-req-build-d9c7zi3u
  Created wheel for model-constructor: filename=model_constructor-0.1.7-cp36-none-any.whl size=21562 sha256=2a433cd6c26cbbfbc2482eb3669b1fcda074317341a666d8ff377f617ec23235
  Stored in directory: /tmp/pip-ephem-wheel-cache-nedz39oc/wheels/5b/92/65/8093a307d5802f41f4a8776b40bc12b558e75a2a906ae8b683
Successfully built model-constructor


In [ ]:
!pip install git+https://github.com/ayasyrev/imagenette_experiments

  Cloning https://github.com/ayasyrev/imagenette_experiments to /tmp/pip-req-build-xxb7rcr8
  Running command git clone -q https://github.com/ayasyrev/imagenette_experiments /tmp/pip-req-build-xxb7rcr8
  Created wheel for imagenette-experiments: filename=imagenette_experiments-0.0.1-cp36-none-any.whl size=15279 sha256=8b503d138eeff69f12cd02b8b0d78c89c547b35989d30181c29d3f83444e8803
  Stored in directory: /tmp/pip-ephem-wheel-cache-f7j3vj4p/wheels/af/99/98/2212941f45a18bf6d191f56c39e3569138414560c76defc0d4
Successfully built imagenette-experiments


In [ ]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *

In [ ]:
# pip install kornia
from kornia.contrib import MaxBlurPool2d

In [ ]:
from imagenette_experiments.train_utils import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# ResBlock

In [ ]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ks==3 and groups==1:  # to be used for the "stem" of ResNet
          # if ni==3: stride = 2
          layers = [('Conv3x3', nn.Conv2d(ni, ni*dm, 3, stride=stride, padding=1, bias=bias, groups=ni)),
                    ('Conv1x1', nn.Conv2d(ni*dm, nf, 1, bias=bias, groups=1))]
        else:
          layers = [('Conv{}x{}'.format(ks,ks), 
                      nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [ ]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, groups=1):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [# (f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(ni, nf, 3, zero_bn=zero_bn, act=False, bn_layer=True))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh*dm, 3, groups=nh, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*dm, 3, groups=nh, act=False, bn_layer=False)),
                   (f"conv_2", conv_layer(nh*dm, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [ ]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True

## Experiment

In [ ]:
dm = 4
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpola

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


KeyboardInterrupt: ignored

Start with a model with `*2` in all the depthwise convolutions.

In [ ]:
epochs = 60
mixup = 0 if epochs<=20 else 0.2
learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
print(learn.recorder.metrics[-1][0].item())

In [ ]:
learn.validate()

In [ ]:
state = learn.model.state_dict()
channels = 0
for name in state:
  if 'Conv3x3' in name and channels==0:
    a,b,c,d = state[name].size()
    double = state[name].unsqueeze(1).expand(a,2,b,c,d).reshape(a*2,b,c,d)
    state[name] = double / 1.414
    channels = a
  if 'Conv1x1' in name and state[name].size()[1]==channels:
    a,b,c,d = state[name].size()
    double = state[name].unsqueeze(2).expand(a,b,2,c,d).reshape(a,b*2,c,d)
    state[name] = double / 1.414
    channels = 0
print('done doubling')

Here, manually change the `*2` to `*4` (in four places), rerun `model`, and proceed below:

In [ ]:
epochs = 10
mixup = 0 if epochs<=20 else 0.2
learn2 = get_learn(model=model, size=192, bs=16, mixup=mixup)
learn2.model.load_state_dict(state)
print(learn2.validate())

In [ ]:
learn2.fit_fc(10, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
print(learn2.recorder.metrics[-1][0].item())

In [ ]:
learn.validate()

In [ ]:
learn2.fit_fc(40, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)

In [ ]:
for i in range(40):
  print(i, learn2.recorder.val_losses[i].item(), learn2.recorder.metrics[i][0].item())

# results

## size=128


In [ ]:
dm = 4
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print({e: sum(res[e])/len(res[e]) for e in res})

In [ ]:
print('depth multiplier =',dm)
print({e: sum(res[e])/len(res[e]) for e in res})

In [ ]:
dm = 6
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier='.format(dm), {e: sum(res[e])/len(res[e])})

In [ ]:
res[-5:], sum(res[-5:])/5

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
res[5:10], sum(res[5:10])/5

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
res = 
for epochs in [5]*5 + [20]*5 + [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

### epochs=5, 5 runs

no stride, bs=16, both act and bn

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

### epochs=20, 5 runs

no stride, bs=16, both act and bn

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

### epochs=80, 1 run

no stride, bs=16, no act, no bn

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))


no stride, bs=16, both act and bn

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

## size=192

In [ ]:
dm = 2
res = dict()
for e in [5,80]: # [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

In [ ]:
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

In [ ]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

In [ ]:
state = learn.model.state_dict()

In [ ]:
torch.save(state, 'drive/app/ResNet50_depth.pth')

In [ ]:
from google.colab import files
files.download('./checkpoint/ResNet50_depth.pth')

In [ ]:
dm = 2
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

experimenting with channel multiplier

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
res



### epochs=5, 5 runs

no stride, bs=16, act=False, bn=False

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

no stride, bs=16, act=True, bn=False

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

### epochs=20, 5 runs

no stride, bs=16, no act no bn

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

### epochs=80, 1 run

no stride, bs=16, no act no bn

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

Sorry, data was "truncated", but see the val_loss and accuracy below for proof. **92.08 is the all-time best that I've seen for Imagewoof2.**

In [ ]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

### epochs=200, 1 run

no stride at stem, bs=16, no act no bn

In [ ]:
res = []
for epochs in [200]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
res

In [ ]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

## size=256

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))